In [7]:
import os
import time
import json
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.table import Table

# Specify the directory containing JSON files
json_directory = "./jsonFiles"

# Specify the directory to save the reports
reports_directory = "./admin-chart"

# Keep track of processed files to avoid reprocessing
processed_files = set()

# Function to create and save a table chart for a given order
def create_and_save_table(order_data):
    # Create a new figure
    plt.figure(figsize=(8, 4))

    # Create a table to display the values
    data = [['Order ID', 'Date', 'Order Count', 'Product Count', 'Total Revenue']]
    data.extend(order_data)

    table = plt.table(cellText=data, loc='center', cellLoc='center', cellColours=[['lightgrey']*5]*(len(order_data) + 1), colColours=['lightgrey']*5)
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    # Remove axis labels and ticks
    plt.axis('off')

    # Save the chart as an image file
    chart_filename = os.path.join(reports_directory, f"{order_data[0][0]}.png")
    plt.savefig(chart_filename)
    plt.close('all')

# Continuously monitor the directory for new JSON files
while True:
    existing_files = [os.path.join(json_directory, filename) for filename in os.listdir(json_directory) if filename.endswith(".json")]
    new_files = [file for file in existing_files if file not in processed_files]

    for filename in new_files:
        with open(filename, 'r') as file:
            json_str = file.read()
            try:
                data = json.loads(json_str)
                order_data = []

                for item in data:
                    if isinstance(item, dict):  # Check if item is a dictionary
                        order_id = item.get("_id", {}).get("$oid", "N/A")
                        order_count = 0
                        product_count = {}
                        total_revenue = 0

                        # Extract item details
                        cart_items = item.get("cartItems", [])
                        for cart_item in cart_items:
                            quantity = cart_item.get("quantity", 0)
                            price = cart_item.get("price", 0)

                            # Update total revenue
                            total_revenue += price * quantity

                            # Update product count
                            product_count["Product ID"] = product_count.get("Product ID", 0) + quantity

                        # Update order count
                        order_count = len(cart_items)

                        order_data.append([order_id, str(date.today()), order_count, sum(product_count.values()), total_revenue])
                        processed_files.add(filename)
                        print("Updated reports with data from:", filename)

                # Create and save a table chart for each order
                create_and_save_table(order_data)

            except json.JSONDecodeError:
                print(f"Error decoding JSON in file: {filename}")

    time.sleep(60)  # Sleep for 60 seconds before checking again (e.g., every minute)


AttributeError: 'str' object has no attribute 'get'